# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # text to bag of words = vocab
    
    vocab = set(text)
    
    vocab2int = {word: idx for idx, word in enumerate(vocab)}
    int2vocab = dict(enumerate(vocab))
    
    return vocab2int, int2vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    lookup_dict = {
        '.': '<period>',
        ',': '<comma>',
        '"': '<doublequote>',
        ';': '<semicolon>',
        '!': '<exclamation>',
        '?': '<question>',
        '(': '<leftparen>',
        ')': '<rightparen>',
        '--': '<dash>',
        '\n': '<return>'
    }
    return lookup_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    input = tf.placeholder(tf.int32, shape=(None, None), name='input')
    targets = tf.placeholder(tf.int32, shape=(None, None), name='targets')
    lrate = tf.placeholder(tf.float32, shape=(None), name='lrate')
    return input, targets, lrate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [32]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """

    num_layers = 2
    keep_prob = 0.95
    
    def lstm_cell():
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size, activation=tf.sigmoid)
        lstm = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm
    
    rnn = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(num_layers)])
    state = tf.identity(rnn.zero_state(batch_size, tf.float32), name='initial_state')
    
    return rnn, state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [33]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """

    embedding = tf.Variable(
        initial_value=tf.random_uniform(shape=(vocab_size, embed_dim), minval=-1, maxval=1),
        name='embedding')

    emb = tf.nn.embedding_lookup(embedding, input_data)

    return emb


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [34]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name='final_state')

    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [35]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    emb_input = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell=cell, inputs=emb_input)
    
    # according to cs231n recommendation
    print(input_data.get_shape().as_list())
    stddev_ = 0.1 if input_data.get_shape().as_list()[0] is None else 2.0/np.sqrt(np.prod(input_data.get_shape().as_list()[1:]))
    print('stddev = ', stddev_)
    
    weights_init = tf.truncated_normal_initializer(stddev=stddev_)
    bias_init = tf.zeros_initializer()

    logits = tf.contrib.layers.fully_connected(
        inputs=outputs,
        num_outputs=vocab_size,
        activation_fn=None,
        weights_initializer=weights_init,
        biases_initializer=bias_init)
    
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

[128, 5]
stddev =  0.894427191
Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [36]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    n_words = batch_size * seq_length
    n_batches = int(len(int_text) / n_words)
                    
    inputs = np.array(int_text[0:n_batches * n_words])
    targets = np.array(int_text[1:n_batches * n_words + 1])
                    
    input_batches = np.split(inputs.reshape(batch_size, -1), n_batches, 1)
    target_batches = np.split(targets.reshape(batch_size, -1), n_batches, 1)
    
    output = np.array(list(zip(input_batches, target_batches)))
    output[-1,-1,-1,-1] = output[0,0,0,0]
                    
    return output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [41]:
# Number of Epochs
num_epochs = 500
# Batch Size
batch_size = 64
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 16
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 20

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [42]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

[None, None]
stddev =  0.1


## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [43]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/67   train_loss = 9.172
Epoch   0 Batch   20/67   train_loss = 6.186
Epoch   0 Batch   40/67   train_loss = 6.174
Epoch   0 Batch   60/67   train_loss = 5.718
Epoch   1 Batch   13/67   train_loss = 5.526
Epoch   1 Batch   33/67   train_loss = 5.377
Epoch   1 Batch   53/67   train_loss = 5.599
Epoch   2 Batch    6/67   train_loss = 5.194
Epoch   2 Batch   26/67   train_loss = 5.078
Epoch   2 Batch   46/67   train_loss = 5.117
Epoch   2 Batch   66/67   train_loss = 4.957
Epoch   3 Batch   19/67   train_loss = 4.969
Epoch   3 Batch   39/67   train_loss = 4.860
Epoch   3 Batch   59/67   train_loss = 4.907
Epoch   4 Batch   12/67   train_loss = 4.757
Epoch   4 Batch   32/67   train_loss = 4.934
Epoch   4 Batch   52/67   train_loss = 4.926
Epoch   5 Batch    5/67   train_loss = 4.827
Epoch   5 Batch   25/67   train_loss = 4.713
Epoch   5 Batch   45/67   train_loss = 4.671
Epoch   5 Batch   65/67   train_loss = 4.423
Epoch   6 Batch   18/67   train_loss = 4.643
Epoch   6 

Epoch  54 Batch   42/67   train_loss = 1.476
Epoch  54 Batch   62/67   train_loss = 1.541
Epoch  55 Batch   15/67   train_loss = 1.491
Epoch  55 Batch   35/67   train_loss = 1.516
Epoch  55 Batch   55/67   train_loss = 1.474
Epoch  56 Batch    8/67   train_loss = 1.555
Epoch  56 Batch   28/67   train_loss = 1.392
Epoch  56 Batch   48/67   train_loss = 1.502
Epoch  57 Batch    1/67   train_loss = 1.548
Epoch  57 Batch   21/67   train_loss = 1.478
Epoch  57 Batch   41/67   train_loss = 1.477
Epoch  57 Batch   61/67   train_loss = 1.429
Epoch  58 Batch   14/67   train_loss = 1.459
Epoch  58 Batch   34/67   train_loss = 1.503
Epoch  58 Batch   54/67   train_loss = 1.490
Epoch  59 Batch    7/67   train_loss = 1.467
Epoch  59 Batch   27/67   train_loss = 1.402
Epoch  59 Batch   47/67   train_loss = 1.495
Epoch  60 Batch    0/67   train_loss = 1.472
Epoch  60 Batch   20/67   train_loss = 1.409
Epoch  60 Batch   40/67   train_loss = 1.327
Epoch  60 Batch   60/67   train_loss = 1.382
Epoch  61 

Epoch 109 Batch   17/67   train_loss = 0.572
Epoch 109 Batch   37/67   train_loss = 0.542
Epoch 109 Batch   57/67   train_loss = 0.603
Epoch 110 Batch   10/67   train_loss = 0.569
Epoch 110 Batch   30/67   train_loss = 0.535
Epoch 110 Batch   50/67   train_loss = 0.508
Epoch 111 Batch    3/67   train_loss = 0.543
Epoch 111 Batch   23/67   train_loss = 0.549
Epoch 111 Batch   43/67   train_loss = 0.531
Epoch 111 Batch   63/67   train_loss = 0.547
Epoch 112 Batch   16/67   train_loss = 0.504
Epoch 112 Batch   36/67   train_loss = 0.501
Epoch 112 Batch   56/67   train_loss = 0.539
Epoch 113 Batch    9/67   train_loss = 0.514
Epoch 113 Batch   29/67   train_loss = 0.492
Epoch 113 Batch   49/67   train_loss = 0.545
Epoch 114 Batch    2/67   train_loss = 0.520
Epoch 114 Batch   22/67   train_loss = 0.524
Epoch 114 Batch   42/67   train_loss = 0.430
Epoch 114 Batch   62/67   train_loss = 0.483
Epoch 115 Batch   15/67   train_loss = 0.473
Epoch 115 Batch   35/67   train_loss = 0.488
Epoch 115 

Epoch 163 Batch   59/67   train_loss = 0.311
Epoch 164 Batch   12/67   train_loss = 0.314
Epoch 164 Batch   32/67   train_loss = 0.298
Epoch 164 Batch   52/67   train_loss = 0.303
Epoch 165 Batch    5/67   train_loss = 0.259
Epoch 165 Batch   25/67   train_loss = 0.313
Epoch 165 Batch   45/67   train_loss = 0.286
Epoch 165 Batch   65/67   train_loss = 0.273
Epoch 166 Batch   18/67   train_loss = 0.282
Epoch 166 Batch   38/67   train_loss = 0.268
Epoch 166 Batch   58/67   train_loss = 0.277
Epoch 167 Batch   11/67   train_loss = 0.270
Epoch 167 Batch   31/67   train_loss = 0.325
Epoch 167 Batch   51/67   train_loss = 0.329
Epoch 168 Batch    4/67   train_loss = 0.274
Epoch 168 Batch   24/67   train_loss = 0.279
Epoch 168 Batch   44/67   train_loss = 0.256
Epoch 168 Batch   64/67   train_loss = 0.277
Epoch 169 Batch   17/67   train_loss = 0.309
Epoch 169 Batch   37/67   train_loss = 0.284
Epoch 169 Batch   57/67   train_loss = 0.305
Epoch 170 Batch   10/67   train_loss = 0.276
Epoch 170 

Epoch 218 Batch   34/67   train_loss = 0.255
Epoch 218 Batch   54/67   train_loss = 0.229
Epoch 219 Batch    7/67   train_loss = 0.262
Epoch 219 Batch   27/67   train_loss = 0.227
Epoch 219 Batch   47/67   train_loss = 0.246
Epoch 220 Batch    0/67   train_loss = 0.244
Epoch 220 Batch   20/67   train_loss = 0.278
Epoch 220 Batch   40/67   train_loss = 0.258
Epoch 220 Batch   60/67   train_loss = 0.254
Epoch 221 Batch   13/67   train_loss = 0.225
Epoch 221 Batch   33/67   train_loss = 0.210
Epoch 221 Batch   53/67   train_loss = 0.232
Epoch 222 Batch    6/67   train_loss = 0.239
Epoch 222 Batch   26/67   train_loss = 0.274
Epoch 222 Batch   46/67   train_loss = 0.222
Epoch 222 Batch   66/67   train_loss = 0.253
Epoch 223 Batch   19/67   train_loss = 0.223
Epoch 223 Batch   39/67   train_loss = 0.255
Epoch 223 Batch   59/67   train_loss = 0.264
Epoch 224 Batch   12/67   train_loss = 0.276
Epoch 224 Batch   32/67   train_loss = 0.247
Epoch 224 Batch   52/67   train_loss = 0.245
Epoch 225 

Epoch 273 Batch    9/67   train_loss = 0.221
Epoch 273 Batch   29/67   train_loss = 0.213
Epoch 273 Batch   49/67   train_loss = 0.242
Epoch 274 Batch    2/67   train_loss = 0.221
Epoch 274 Batch   22/67   train_loss = 0.222
Epoch 274 Batch   42/67   train_loss = 0.209
Epoch 274 Batch   62/67   train_loss = 0.233
Epoch 275 Batch   15/67   train_loss = 0.213
Epoch 275 Batch   35/67   train_loss = 0.244
Epoch 275 Batch   55/67   train_loss = 0.228
Epoch 276 Batch    8/67   train_loss = 0.214
Epoch 276 Batch   28/67   train_loss = 0.201
Epoch 276 Batch   48/67   train_loss = 0.215
Epoch 277 Batch    1/67   train_loss = 0.193
Epoch 277 Batch   21/67   train_loss = 0.226
Epoch 277 Batch   41/67   train_loss = 0.233
Epoch 277 Batch   61/67   train_loss = 0.219
Epoch 278 Batch   14/67   train_loss = 0.204
Epoch 278 Batch   34/67   train_loss = 0.229
Epoch 278 Batch   54/67   train_loss = 0.214
Epoch 279 Batch    7/67   train_loss = 0.242
Epoch 279 Batch   27/67   train_loss = 0.213
Epoch 279 

Epoch 327 Batch   51/67   train_loss = 0.264
Epoch 328 Batch    4/67   train_loss = 0.222
Epoch 328 Batch   24/67   train_loss = 0.213
Epoch 328 Batch   44/67   train_loss = 0.194
Epoch 328 Batch   64/67   train_loss = 0.208
Epoch 329 Batch   17/67   train_loss = 0.260
Epoch 329 Batch   37/67   train_loss = 0.233
Epoch 329 Batch   57/67   train_loss = 0.225
Epoch 330 Batch   10/67   train_loss = 0.202
Epoch 330 Batch   30/67   train_loss = 0.195
Epoch 330 Batch   50/67   train_loss = 0.199
Epoch 331 Batch    3/67   train_loss = 0.204
Epoch 331 Batch   23/67   train_loss = 0.239
Epoch 331 Batch   43/67   train_loss = 0.238
Epoch 331 Batch   63/67   train_loss = 0.219
Epoch 332 Batch   16/67   train_loss = 0.219
Epoch 332 Batch   36/67   train_loss = 0.219
Epoch 332 Batch   56/67   train_loss = 0.247
Epoch 333 Batch    9/67   train_loss = 0.203
Epoch 333 Batch   29/67   train_loss = 0.211
Epoch 333 Batch   49/67   train_loss = 0.232
Epoch 334 Batch    2/67   train_loss = 0.219
Epoch 334 

Epoch 382 Batch   26/67   train_loss = 0.249
Epoch 382 Batch   46/67   train_loss = 0.192
Epoch 382 Batch   66/67   train_loss = 0.199
Epoch 383 Batch   19/67   train_loss = 0.197
Epoch 383 Batch   39/67   train_loss = 0.244
Epoch 383 Batch   59/67   train_loss = 0.240
Epoch 384 Batch   12/67   train_loss = 0.237
Epoch 384 Batch   32/67   train_loss = 0.224
Epoch 384 Batch   52/67   train_loss = 0.216
Epoch 385 Batch    5/67   train_loss = 0.202
Epoch 385 Batch   25/67   train_loss = 0.218
Epoch 385 Batch   45/67   train_loss = 0.217
Epoch 385 Batch   65/67   train_loss = 0.214
Epoch 386 Batch   18/67   train_loss = 0.215
Epoch 386 Batch   38/67   train_loss = 0.202
Epoch 386 Batch   58/67   train_loss = 0.206
Epoch 387 Batch   11/67   train_loss = 0.211
Epoch 387 Batch   31/67   train_loss = 0.251
Epoch 387 Batch   51/67   train_loss = 0.260
Epoch 388 Batch    4/67   train_loss = 0.216
Epoch 388 Batch   24/67   train_loss = 0.210
Epoch 388 Batch   44/67   train_loss = 0.187
Epoch 388 

Epoch 437 Batch    1/67   train_loss = 0.178
Epoch 437 Batch   21/67   train_loss = 0.213
Epoch 437 Batch   41/67   train_loss = 0.212
Epoch 437 Batch   61/67   train_loss = 0.207
Epoch 438 Batch   14/67   train_loss = 0.199
Epoch 438 Batch   34/67   train_loss = 0.220
Epoch 438 Batch   54/67   train_loss = 0.197
Epoch 439 Batch    7/67   train_loss = 0.240
Epoch 439 Batch   27/67   train_loss = 0.184
Epoch 439 Batch   47/67   train_loss = 0.204
Epoch 440 Batch    0/67   train_loss = 0.220
Epoch 440 Batch   20/67   train_loss = 0.245
Epoch 440 Batch   40/67   train_loss = 0.209
Epoch 440 Batch   60/67   train_loss = 0.208
Epoch 441 Batch   13/67   train_loss = 0.207
Epoch 441 Batch   33/67   train_loss = 0.188
Epoch 441 Batch   53/67   train_loss = 0.167
Epoch 442 Batch    6/67   train_loss = 0.206
Epoch 442 Batch   26/67   train_loss = 0.231
Epoch 442 Batch   46/67   train_loss = 0.178
Epoch 442 Batch   66/67   train_loss = 0.188
Epoch 443 Batch   19/67   train_loss = 0.189
Epoch 443 

Epoch 491 Batch   43/67   train_loss = 0.220
Epoch 491 Batch   63/67   train_loss = 0.234
Epoch 492 Batch   16/67   train_loss = 0.204
Epoch 492 Batch   36/67   train_loss = 0.207
Epoch 492 Batch   56/67   train_loss = 0.228
Epoch 493 Batch    9/67   train_loss = 0.209
Epoch 493 Batch   29/67   train_loss = 0.193
Epoch 493 Batch   49/67   train_loss = 0.227
Epoch 494 Batch    2/67   train_loss = 0.217
Epoch 494 Batch   22/67   train_loss = 0.223
Epoch 494 Batch   42/67   train_loss = 0.189
Epoch 494 Batch   62/67   train_loss = 0.234
Epoch 495 Batch   15/67   train_loss = 0.167
Epoch 495 Batch   35/67   train_loss = 0.233
Epoch 495 Batch   55/67   train_loss = 0.214
Epoch 496 Batch    8/67   train_loss = 0.204
Epoch 496 Batch   28/67   train_loss = 0.181
Epoch 496 Batch   48/67   train_loss = 0.192
Epoch 497 Batch    1/67   train_loss = 0.177
Epoch 497 Batch   21/67   train_loss = 0.211
Epoch 497 Batch   41/67   train_loss = 0.208
Epoch 497 Batch   61/67   train_loss = 0.202
Epoch 498 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [44]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [45]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [46]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    inputs = loaded_graph.get_tensor_by_name('input:0')
    initial_state = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state = loaded_graph.get_tensor_by_name('final_state:0')
    probs = loaded_graph.get_tensor_by_name('probs:0')
    
    return inputs, initial_state, final_state, probs

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [48]:
import random
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    rand = random.uniform(0, 1.0)
    cum_prob = 0
    for i, word in enumerate(int_to_vocab):
        cum_prob += probabilities[i]
        if (rand <= cum_prob):
            break
            
    return int_to_vocab[word]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [49]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak: yeah, yeah, i've been meanin' to get that updated, uh, for this... we need the kind of" tell that on, walking in the house of getting the next one could ever have a beer?
lenny_leonard: sure beats a tax of my drink.
moe_szyslak: really?
homer_simpson: yeah, they're italian.
carl_carlson: ya still?
lisa_simpson:(thinking of pain) 'cause he don't drink, huh?
moe_szyslak: i wouldn't-a had none of this without you, kid.
moe_szyslak: i got just the drink to wipe your mind clean... the uh, is that the booze in this world. i'm talkin' malfeasance here.
artie_ziff:(sincere) see everybody! i'm not your clown! don't diminish me!
homer_simpson: gentlemen, i bid you if you were hilarious.
moe_szyslak: i was just doin' what comes naturally to drink my money.
lenny_leonard: you could save mckinley.
homer_simpson: it's not a guy who get three-- from now on, we're watching for us a lucky woman.


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.